In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [6]:
submission_key = pd.read_csv('submission_key.csv')
historical_weather = pd.read_csv('historical_weather.csv')


In [7]:
historical_weather.head()

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh
0,C001,1/1/2014,6.6,-1.4,11.6,NaN,NaN,168.0,6.2
1,C001,1/2/2014,9.3,6.3,13.3,NaN,NaN,155.0,10.0
2,C001,1/3/2014,7.6,1.9,14.0,NaN,NaN,NaN,5.8
3,C001,1/4/2014,7.6,3.9,13.3,NaN,NaN,291.0,11.3
4,C001,1/5/2014,8.6,0.5,16.9,NaN,NaN,NaN,5.0


In [8]:
submission_key.head()

,submission_ID,city_id,date
0,1,C001,1/1/2019
1,2,C001,1/2/2019
2,3,C001,1/3/2019
3,4,C001,1/4/2019
4,5,C001,1/5/2019


In [9]:
historical_weather.isnull().sum()

city_id                    0
date                       0
avg_temp_c              1224
min_temp_c              5886
max_temp_c              7493
precipitation_mm       69744
snow_depth_mm         170100
avg_wind_dir_deg       35394
avg_wind_speed_kmh     22472
dtype: int64

In [47]:
historical_weather = pd.read_csv('historical_weather.csv')

# Convert 'city_id' to categorical and then to numeric
historical_weather['city_id'] = historical_weather['city_id'].astype('category')
historical_weather['city_id'] = historical_weather['city_id'].cat.codes

# Create new features
historical_weather['date'] = pd.to_datetime(historical_weather['date'])
historical_weather['day_of_year'] = historical_weather['date'].dt.dayofyear

# fill null snow_depth_mm, precipitation to -1
historical_weather['snow_depth_mm'] = historical_weather['snow_depth_mm'].fillna(-1)
historical_weather['precipitation_mm'] = historical_weather['precipitation_mm'].fillna(-1)

# Lag features, snow_depth_mm and precipitation_mm
historical_weather['snow_depth_mm_lag_7'] = historical_weather['snow_depth_mm'].shift(7)
historical_weather['precipitation_mm_lag_7'] = historical_weather['precipitation_mm'].shift(7)

# Lag features: Shift the avg_temp_c column by 7 days to get last week's average temperature
historical_weather['avg_temp_lag_7'] = historical_weather['avg_temp_c'].shift(7)

# Rolling averages: Rolling average of the last 7 days
historical_weather['avg_temp_roll_7'] = historical_weather['avg_temp_c'].rolling(window=7).mean()

# Step 2: Create a column for the previous year's date
for year in range(1, 4):
  historical_weather[f'lag_{year}y_date'] = historical_weather['date'] - pd.DateOffset(years=year)

  # Step 3: Merge historical_weather with itself to bring in last year's data
  # Note: Assuming 'city_id' and 'date' uniquely identify rows and you want to bring in 'avg_temp_c' from last year
  historical_weather = historical_weather.merge(
    historical_weather[['city_id', 'date', 'avg_temp_c', 'min_temp_c', 'max_temp_c']],
    left_on=['city_id', f'lag_{year}y_date'],
    right_on=['city_id', 'date'],
    suffixes=('', f'_lag_{year}y'),
    how='left'
  )
  historical_weather = historical_weather.drop(f'date_lag_{year}y', axis=1)
  historical_weather = historical_weather.drop(f'lag_{year}y_date', axis=1)

# fill in NaNs
for column in historical_weather.columns.drop('date'):
    if historical_weather[column].dtype != 'object':
        # Fill NaNs with the average of the value before and after
        historical_weather[column] = historical_weather[column].interpolate(method='linear')

# Step 4: Filter to include only rows after the specified date
# This step is already done with filtered_weather, but if you need to apply it again after the merge:
filtered_historical_weather = historical_weather[historical_weather['date'] > pd.Timestamp('2017-01-01')]

In [48]:
# Import necessary libraries for modeling
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



# Define the target variable and features
X = filtered_historical_weather.drop(
    columns=[
        'date',
        'avg_temp_c',
        'min_temp_c',
        'max_temp_c',
        'precipitation_mm',
        'snow_depth_mm',
        'avg_wind_dir_deg',
        'avg_wind_speed_kmh',
        'snow_depth_mm_lag_7',
        'precipitation_mm_lag_7'
        ]
    )
y = filtered_historical_weather['avg_temp_c']
print(X.head())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


      city_id  day_of_year  avg_temp_lag_7  avg_temp_roll_7  \
1097        0            2            13.8        10.942857   
1098        0            3            10.7        11.171429   
1099        0            4            11.8        10.928571   
1100        0            5            12.8        10.542857   
1101        0            6            12.6        10.285714   

      avg_temp_c_lag_1y  min_temp_c_lag_1y  max_temp_c_lag_1y  \
1097                2.7               -5.5               14.4   
1098                3.2               -4.7               14.9   
1099                5.2               -2.9               15.1   
1100                6.6                1.5               13.2   
1101                4.8               -0.6                9.1   

      avg_temp_c_lag_2y  min_temp_c_lag_2y  max_temp_c_lag_2y  \
1097                3.2               -4.2                9.3   
1098                5.4                2.3                9.2   
1099                2.5            

In [55]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RandomForestRegressor RMSE: {rmse}')

RandomForestRegressor RMSE: 1.9713788553032423


In [49]:
from sklearn.linear_model import LinearRegression

# Create a Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Evaluate the model
y_pred_linear = linear_model.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
print(f'Linear Regression RMSE: {rmse_linear}')


Linear Regression RMSE: 1.9958266586070574


In [88]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# scaling for SGD
scaler = StandardScaler()
scaler.fit(X)
Xs_train = scaler.transform(X_train)
Xs_test = scaler.transform(X_test)

# Create a SGDRegressor model
sgd = SGDRegressor()
sgd.fit(Xs_train, y_train)

# Evaluate the model
y_pred_SGD = sgd.predict(Xs_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_SGD))
print(f'SGDRegressor RMSE: {rmse_linear}')

SGDRegressor RMSE: 1.9969624364936764


In [91]:
# OUTPUT
submission_key = pd.read_csv('submission_key.csv')

submission_key['city_id'] = submission_key['city_id'].astype('category')
original_categories = submission_key['city_id'].cat.categories
code_to_category_mapping = dict(enumerate(original_categories))
submission_key['city_id'] = submission_key['city_id'].cat.codes

submission_key['date'] = pd.to_datetime(submission_key['date'])
submission_key['day_of_year'] = submission_key['date'].dt.dayofyear

def lag_submission_avg_temp(row):
  return row

# Lag features: Shift the avg_temp_c column by 7 days to get last week's average temperature
historical_weather['avg_temp_lag_7'] = historical_weather['avg_temp_c'].shift(7)

# Rolling averages: Rolling average of the last 7 days
historical_weather['avg_temp_roll_7'] = historical_weather['avg_temp_c'].rolling(window=7).mean()

# add avg_temp_c last week
submission_key['date_last_week'] = submission_key['date'] - pd.Timedelta(days=7)

submission_key = submission_key.merge(
    historical_weather[['city_id', 'date', 'avg_temp_c']],
    left_on=['city_id', 'date_last_week'],
    right_on=['city_id', 'date'],
    suffixes=('', '_last_week'),
    how='left'
).drop('date_last_week', axis=1)

submission_key = submission_key.rename(columns={'avg_temp_c': 'avg_temp_lag_7'})

# add rolling average
submission_key['date_last_week'] = submission_key['date'] - pd.Timedelta(days=7)

# Merge to get historical data for the past week
merged_data = submission_key.merge(
    historical_weather[['city_id', 'date', 'avg_temp_c']],
    left_on=['city_id', 'date_last_week'],
    right_on=['city_id', 'date'],
    how='left'
)

# Calculate rolling average for the past week
merged_data['avg_temp_roll_7'] = (
    merged_data
    .groupby('city_id')['avg_temp_c']
    .rolling(window=7, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# Merge rolling average back into submission_key
submission_key = submission_key.merge(
    merged_data[['city_id', 'date_last_week', 'avg_temp_roll_7']],
    on=['city_id', 'date_last_week'],
    how='left'
).drop('date_last_week', axis=1)


# Step 2: Create a column for the previous year's date
for year in range(1, 4):
  submission_key[f'lag_{year}y_date'] = submission_key['date'] - pd.DateOffset(years=year)

  # Step 3: Merge historical_weather with itself to bring in last year's data
  # Note: Assuming 'city_id' and 'date' uniquely identify rows and you want to bring in 'avg_temp_c' from last year
  submission_key = submission_key.merge(
    historical_weather[['city_id', 'date', 'avg_temp_c', 'min_temp_c', 'max_temp_c']],
    left_on=['city_id', f'lag_{year}y_date'],
    right_on=['city_id', 'date'],
    suffixes=('', f'_lag_{year}y'),
    how='left'
  )
  submission_key = submission_key.drop(f'date_lag_{year}y', axis=1)
  submission_key = submission_key.drop(f'lag_{year}y_date', axis=1)

submission_key = submission_key.rename(
    columns={
        'avg_temp_c': 'avg_temp_c_lag_1y',
        'min_temp_c': 'min_temp_c_lag_1y',
        'max_temp_c': 'max_temp_c_lag_1y',
    }
)

print(submission_key.head())
y_pred = model.predict(submission_key.drop('submission_ID', axis=1).drop('date', axis=1))
output = pd.DataFrame(
    {
      'submission_ID': submission_key['submission_ID'],
      'city_id': submission_key['city_id'].map(code_to_category_mapping),
      # convert date to m/dd/yyyy but remove the 0s
      'date': submission_key['date'].dt.strftime('%-m/%-d/%Y'),
      'avg_temp_c': y_pred
    }
  )
print(output.head())


output.to_csv('submission.csv', index=False)

   submission_ID  city_id       date  day_of_year  avg_temp_lag_7  \
0              1        0 2019-01-01            1             9.0   
1              2        0 2019-01-02            2             9.4   
2              3        0 2019-01-03            3             9.7   
3              4        0 2019-01-04            4            10.1   
4              5        0 2019-01-05            5             8.9   

   avg_temp_roll_7  avg_temp_c_lag_1y  min_temp_c_lag_1y  max_temp_c_lag_1y  \
0         9.000000                8.1                2.0               17.0   
1         9.200000                8.5                1.0               19.0   
2         9.366667                9.6                2.0               17.0   
3         9.550000               11.8                9.0               16.0   
4         9.420000               11.6                6.0               19.0   

   avg_temp_c_lag_2y  min_temp_c_lag_2y  max_temp_c_lag_2y  avg_temp_c_lag_3y  \
0                8.2         